<a href="https://colab.research.google.com/github/lupis30puc/yelp_bert_random_forest/blob/update-5/Yelp_BERT_feature_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Set Up

In [ ]:
!pip install treeinterpreter
#https://pypi.org/project/treeinterpreter/

In [ ]:
!pip install transformers

     |████████████████████████████████| 1.8MB 4.2MB/s 
     |████████████████████████████████| 2.9MB 23.6MB/s 
     |████████████████████████████████| 890kB 34.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=d2ccb411975161c32ac26202313a8e9cf93bb4861159d2d6158694d4bb0b5a3d
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import string
import math

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, roc_curve
%matplotlib inline
import time
import pickle
from treeinterpreter import treeinterpreter as ti
import joblib
from transformers import BertTokenizer

## Initializing the models

In [ ]:
# load, no need to initialize the loaded_rf
rf_1000 = joblib.load("/content/drive/MyDrive/Yelp/model_99/rf_est1000_leaf1.joblib")
rf_300 = joblib.load("/content/drive/MyDrive/Yelp/model_99/rf_est300_leaf3.joblib")
rf_grid = joblib.load("/content/drive/MyDrive/Yelp/model_99/rf_grid_search.joblib")

In [ ]:
rf_grid.best_params_  # Parameter setting that gave the best results on the hold out data.
print(rf_grid.best_params_)

{'min_samples_leaf': 2, 'n_estimators': 1000}


In [ ]:
x_train = pd.read_pickle('/content/drive/MyDrive/Yelp/binary_train_rf')
x_test = pd.read_pickle('/content/drive/MyDrive/Yelp/binary_test_rf')

In [ ]:
import torch
y_train = torch.load('/content/drive/MyDrive/Yelp/model_99/flat_pred_labels_train')
y_test  = torch.load('/content/drive/MyDrive/Yelp/model_99/flat_pred_labels_test')

## Comparing the models

In [ ]:
%%time
print("Random Forest 1000 estimators, min leaf 1")
predrf_1000 = rf_1000.predict(x_test)
print("Confusion Matrix for Random Forest Classifier:")
print(confusion_matrix(y_test,predrf_1000))
print("Score:",round(accuracy_score(y_test,predrf_1000)*100,2))
print("Classification Report:")
print(classification_report(y_test,predrf_1000))

Random Forest 1000 estimators, min leaf 1
Confusion Matrix for Random Forest Classifier:
[[1166  115]
 [ 148 1170]]
Score: 89.88
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.91      0.90      1281
           1       0.91      0.89      0.90      1318

    accuracy                           0.90      2599
   macro avg       0.90      0.90      0.90      2599
weighted avg       0.90      0.90      0.90      2599

CPU times: user 3.88 s, sys: 2.62 ms, total: 3.88 s
Wall time: 3.89 s


In [ ]:
print("Test  Accuracy : %f"%rf_1000.score(x_test, y_test))
print("Train Accuracy : %f"%rf_1000.score(x_train, y_train))

Test  Accuracy : 0.90
Train Accuracy : 1.00


In [ ]:
%%time
print("Random Forest 300 estimators, min leaf 3")
predrf_300 = rf_300.predict(x_test)
print("Confusion Matrix for Random Forest Classifier:")
print(confusion_matrix(y_test,predrf_300))
print("Score:",round(accuracy_score(y_test,predrf_300)*100,2))
print("Classification Report:")
print(classification_report(y_test,predrf_300))

Random Forest 300 estimators, min leaf 3
Confusion Matrix for Random Forest Classifier:
[[1160  121]
 [ 151 1167]]
Score: 89.53
Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.91      0.90      1281
           1       0.91      0.89      0.90      1318

    accuracy                           0.90      2599
   macro avg       0.90      0.90      0.90      2599
weighted avg       0.90      0.90      0.90      2599

CPU times: user 613 ms, sys: 5.84 ms, total: 618 ms
Wall time: 627 ms


In [ ]:
print("Test  Accuracy : %f"%rf_300.score(x_test, y_test))
print("Train Accuracy : %f"%rf_300.score(x_train, y_train))

Test  Accuracy : 0.895344
Train Accuracy : 0.951799


In [ ]:
%%time
#rf_grid.best_params_ 
random_f = rf_grid.best_estimator_ 
print("Random Forest 1000 estimators, min leaf 2")
predrf_grid = random_f.predict(x_test)
print("Confusion Matrix for Random Forest Classifier:")
print(confusion_matrix(y_test,predrf_grid))
print("Score:",round(accuracy_score(y_test,predrf_grid)*100,2))
print("Classification Report:")
print(classification_report(y_test,predrf_grid))

Random Forest 1000 estimators, min leaf 2
Confusion Matrix for Random Forest Classifier:
[[1170  111]
 [ 149 1169]]
Score: 90.0
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.91      0.90      1281
           1       0.91      0.89      0.90      1318

    accuracy                           0.90      2599
   macro avg       0.90      0.90      0.90      2599
weighted avg       0.90      0.90      0.90      2599

CPU times: user 1.79 s, sys: 0 ns, total: 1.79 s
Wall time: 1.8 s


In [ ]:
print("Test  Accuracy : %f"%random_f.score(x_test, y_test))
print("Train Accuracy : %f"%random_f.score(x_train, y_train))

Test  Accuracy : 0.899962
Train Accuracy : 0.976814


## Initializing BERT tokenizers and reviews **text**

In [ ]:
# NO NEED FOR THE DATASET OR YES?
train_df = pd.read_pickle('/content/drive/MyDrive/Yelp/sample_train_10394.pkl')
test_df = pd.read_pickle('/content/drive/MyDrive/Yelp/sample_test_2599.pkl')

train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)
#train_df.head()

In [ ]:
# launching the saved model tokenizer
tokenizer = BertTokenizer.from_pretrained('/content/drive/MyDrive/Yelp/model_99/')

In [ ]:
with open('/content/drive/MyDrive/Yelp/tensors_yelp/unique_ids.pkl', 'rb') as f:
  unique_ids = pickle.load( f)

In [ ]:
# Converting unique_ids into words:
%%time
feature_names = [list(tokenizer.vocab.keys())[id] for id in unique_ids]

CPU times: user 39.5 s, sys: 189 ms, total: 39.7 s
Wall time: 40.1 s


In [ ]:
print(len(unique_ids), len(feature_names))

16368 16368


In [ ]:
with open('/content/drive/MyDrive/Yelp/tensors_yelp/feature_names_99.pkl', 'wb') as d:
  pickle.dump(feature_names, d)

## Comparing feature importances

In [ ]:
%%time
importance_300 = rf_300.feature_importances_

CPU times: user 217 ms, sys: 7.02 ms, total: 224 ms
Wall time: 228 ms


In [ ]:
%%time
importance_1000 = rf_1000.feature_importances_

CPU times: user 479 ms, sys: 13 ms, total: 492 ms
Wall time: 493 ms


In [ ]:
%%time
importance_grid = random_f.feature_importances_

CPU times: user 230 ms, sys: 6.94 ms, total: 237 ms
Wall time: 240 ms


In [ ]:
feature_importance_300 = pd.DataFrame({'ids': unique_ids, 'keys': feature_names, 'imp': importance_300})

In [ ]:
feature_importance_1000 = pd.DataFrame({'ids': unique_ids, 'keys': feature_names, 'imp': importance_1000})

In [ ]:
feature_importance_grid = pd.DataFrame({'ids': unique_ids, 'keys': feature_names, 'imp': importance_grid})

In [ ]:
feature_importance_300.head()

,ids,keys,imp
0,0,[PAD],0.002289
1,100,[UNK],0.000010
2,101,[CLS],0.000000
3,102,[SEP],0.000000
4,1037,a,0.000045


In [ ]:
top10_rf300 = list(feature_importance_300.sort_values(by=['imp'], ascending=False)['keys'][:10])
top10_rf300

['great',
 'amazing',
 'delicious',
 'worst',
 'love',
 'asked',
 'horrible',
 'friendly',
 'told',
 'best']

In [ ]:
top10_rf1000 = list(feature_importance_1000.sort_values(by=['imp'], ascending=False)['keys'][:10])
top10_rf1000

['great',
 'delicious',
 'amazing',
 'worst',
 'love',
 'friendly',
 'horrible',
 'told',
 'best',
 'rude']

In [ ]:
top50_rfgrid = list(feature_importance_grid.sort_values(by=['imp'], ascending=False)['keys'][:50])
top50_rfgrid

['great',
 'delicious',
 'amazing',
 'worst',
 'love',
 'friendly',
 'told',
 'horrible',
 'rude',
 'best',
 'asked',
 'terrible',
 'said',
 'minutes',
 'excellent',
 'definitely',
 'awesome',
 'poor',
 'good',
 'bad',
 'bland',
 'favorite',
 'money',
 'perfect',
 'manager',
 'nice',
 'waited',
 'over',
 'loved',
 'left',
 'won',
 'disappointing',
 'un',
 'highly',
 'got',
 'wonderful',
 'fresh',
 'called',
 'slow',
 'awful',
 'tasted',
 'paid',
 'took',
 'ordered',
 'pay',
 'enjoyed',
 'fantastic',
 'order',
 'disappointed',
 'wasn']

## Analyzing with contributions

In [ ]:
%%time
preds_300, bias_300, contributions_300 = ti.predict(rf_300, x_test)
preds_1000, bias_1000, contributions_1000 = ti.predict(rf_1000, x_test)
preds_grid, bias_grid, contributions_grid = ti.predict(random_f, x_test)

CPU times: user 34min 13s, sys: 44.2 s, total: 34min 57s
Wall time: 35min 33s


In [ ]:
preds_300.shape, bias_300.shape, contributions_300.shape

((2599, 2), (2599, 2), (2599, 16368, 2))

In [ ]:
y_test[0]

0

In [ ]:
feature_importance_300[85:100]

,ids,keys,imp
85,1996,the,0.000000e+00
86,1997,of,0.000000e+00
87,1998,and,0.000000e+00
88,1999,in,6.069408e-04
89,2000,to,1.911021e-04
90,2001,was,4.384488e-06
91,2002,he,3.894885e-05
92,2003,is,8.415821e-07
93,2004,as,2.343417e-05
94,2005,for,1.035290e-05


In [ ]:
contributions_300[0][85:100]

array([[ 0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00],
       [-7.96027318e-05,  7.96027318e-05],
       [ 2.81513777e-05, -2.81513777e-05],
       [ 0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00]])

In [ ]:
test_df.clean_text[0]

'positives good location staff reasonably friendly negatives dry manis pedis advertise website update site base coat coat blobbed attention coat cover color nail nails filed totally random lenths shapes shorter end fingers longer pedi woman wasn paying attention ended filing skin bled cute tea menu offer definitely expensive getting answer phone pedi station isn comfortable slightly padded bench notes staff looks surprised sure come ask work'

In [ ]:
# .apply(lambda x : len(x.strip().split(" ")))
len(test_df.clean_text[0].strip().split(" "))

66

In [ ]:
len(test_ids[0])

99

CHECK FROM WHICH COLUMN OF TEST_DF WE ARE GETTING X_TEST FOR BERT... TO GET THE IDS !!!

In [ ]:
with open('/content/drive/MyDrive/Yelp/tensors_yelp/train_ids_99.pkl', 'rb') as f:
    train_ids = pickle.load(f)

with open('/content/drive/MyDrive/Yelp/tensors_yelp/test_ids_99.pkl', 'rb') as d:
    test_ids = pickle.load(d)

In [ ]:
contributions_300[0][0][0]

-0.0021808297309143503

In [ ]:
contributions_300[0][0][1]

0.002180829730914351

In [ ]:
top20[0]

'great'

In [ ]:
w = np.argmax(contributions_300[0][7])
w

1

In [ ]:
w2 = np.argmax(contributions_300[0][9])
w2

0

In [ ]:
contributions_300[0][4]

array([-1.67814874e-05,  1.67814874e-05])

In [ ]:
review_0 = np.nonzero(contributions_300[0])
review_0

(array([    0,     0,     4, ..., 16217, 16306, 16306]),
 array([0, 1, 0, ..., 1, 0, 1]))

In [ ]:
len(review_0[0])

3260

In [ ]:
contributions_300[0].sort()

In [ ]:
contributions_300[0][1]

array([0., 0.])

In [ ]:
if contributions_300[0][1][0] == 0:
  print('works')

works


In [ ]:
len(list(feature_importance_300[feature_importance_300['ids'].isin(test_ids[0])].index))

71

In [ ]:
# from http://blog.datadive.net/random-forest-interpretation-with-scikit-learn/
print("Predictions for random forest 300 and for the review 0")
#top20 = list(feature_importance_300.sort_values(by=['imp'], ascending=False)['keys'][:20])
actual_features = list(feature_importance_300[feature_importance_300['ids'].isin(test_ids[0])]['keys'])
print("Prediction", preds_300[0])
print("Label", np.argmax(preds_300[0], axis=0).flatten())
#print("Bias (trainset prior)", bias_300[0])
print('')
print("Feature contributions:")
actual_features = np.nonzero(contributions_300[0])
#for c, feature in zip(contributions_300[0], top20):
for c, feature in zip(contributions_300[0], top20):
for i in range(len(actual_features)):

  m = np.argmax(c)
  if m == 0:
    me = 'negative'
  else:
    me = 'positive'
  #word = list(tokenizer.vocab.keys())[feature]
  print(feature, c, me)

In [ ]:
# from http://blog.datadive.net/random-forest-interpretation-with-scikit-learn/
print("Predictions for random forest 300 and for the review 0")
#top20 = list(feature_importance_300.sort_values(by=['imp'], ascending=False)['keys'][:20])
#actual_f = list(feature_importance_300[feature_importance_300['ids'].isin(test_ids[0])]['keys'])
con_indx = list(feature_importance_300[feature_importance_300['ids'].isin(test_ids[0])].index)
print("Prediction", preds_300[0])
print("Label", np.argmax(preds_300[0], axis=0).flatten())
#print("Bias (trainset prior)", bias_300[0])
print('')
print("Feature contributions:")
#actual_features = np.nonzero(contributions_300[0])
negative = []
positive = []
for i in con_indx:
  c = contributions_300[0][i]
  feature = feature_importance_300['keys'][i]
  m = np.argmax(c)
  if m == 0:
    negative.append([feature, c])
  else:
    positive.append([feature, c])

  #print(str(feature) + ' has a contribution of: ' \
  #      + str(c) + '\n      thus its meaning is classified as ' + me + '\n ')


print(''+ '\nthe ' + str(len(negative)) + ' contributions for negative are: ')
for l in negative:
  print(str(l[0]) + '       has a contribution of: ' \
        + str(l[1]))
print(''+ '\nthe ' + str(len(positive)) + ' contributions for positive are: ')
for l in positive:
  print(str(l[0]) + '       has a contribution of: ' \
        + str(l[1]))

Predictions for random forest 300 and for the review 0
Prediction [0.51573443 0.48426557]
Label [0]

Feature contributions:

the 56 contributions for negative are: 
[CLS]       has a contribution of: [0. 0.]
[SEP]       has a contribution of: [0. 0.]
b       has a contribution of: [ 0.00265121 -0.00265121]
end       has a contribution of: [ 0.00070298 -0.00070298]
come       has a contribution of: [ 0.00156782 -0.00156782]
station       has a contribution of: [ 0.00126687 -0.00126687]
wasn       has a contribution of: [ 0.0202239 -0.0202239]
woman       has a contribution of: [ 0.00895751 -0.00895751]
site       has a contribution of: [ 0.00275447 -0.00275447]
getting       has a contribution of: [ 0.00964196 -0.00964196]
base       has a contribution of: [0. 0.]
longer       has a contribution of: [ 0.0122008 -0.0122008]
phone       has a contribution of: [ 0.03968604 -0.03968604]
attention       has a contribution of: [ 0.00926254 -0.00926254]
ended       has a contribution of: [ 0.0

In [ ]:
test_df.clean_text[0]

'positives good location staff reasonably friendly negatives dry manis pedis advertise website update site base coat coat blobbed attention coat cover color nail nails filed totally random lenths shapes shorter end fingers longer pedi woman wasn paying attention ended filing skin bled cute tea menu offer definitely expensive getting answer phone pedi station isn comfortable slightly padded bench notes staff looks surprised sure come ask work'

In [ ]:
print("Predictions for random forest 1000 of 2 sample leaves and for the review 0")
con_indx = list(feature_importance_grid[feature_importance_grid['ids'].isin(test_ids[0])].index)
print("Prediction", np.argmax(preds_300[0]))
print("True Label", y_test[0])
print('' + "\nFeature contributions:")
negative = {}
positive = {}
for i in con_indx:
  c = contributions_grid[0][i]
  feature = feature_importance_grid['keys'][i]
  m = np.argmax(c)
  if m == 0:
    negative[feature] = list(c)
  else:
    positive[feature] = list(c)
print(''+ '\nthe ' + str(len(negative)) + ' contributions for negative are: ')
print({key: value[0] for key, value in sorted(negative.items(), key=lambda item: item[1], reverse=True)})
#for l in negative.:
 # print(str(l[0]) + '       has a contribution of: ' \
 #       + str(l[1]))
print(''+ '\nthe ' + str(len(positive)) + ' contributions for positive are: ')
for key, value in sorted(positive.items(), key=lambda item: item[1], reverse=True):
  print(key, value[1])
#for l in positive:
 # print(str(l[0]) + '       has a contribution of: ' \
  #      + str(l[1]))

Predictions for random forest 1000 of 2 sample leaves and for the review 0
Prediction [0.51573443 0.48426557]
Label [0]

Feature contributions:

the 51 contributions for negative are: 
{'phone': 0.03658300672122821, 'dry': 0.029304083900413235, 'wasn': 0.026817347637338878, 'ask': 0.02286670328974162, 'paying': 0.015537179178108115, 'attention': 0.014994646349560426, 'longer': 0.012815955785843582, 'woman': 0.012370776776089696, 'expensive': 0.006366894154202534, 'answer': 0.0057435631050703284, 'getting': 0.00553400806247927, 'ended': 0.00443768852869426, 'negative': 0.0038892178588794354, 'update': 0.003660629653083758, 'looks': 0.0032342630529331668, 'positive': 0.002787047571704352, 'nail': 0.0026753697541031483, 'b': 0.002266570196841662, 'color': 0.0022623463791395194, '##di': 0.0021538215328377637, 'mani': 0.002012021595145529, 'site': 0.0018803509220306216, 'totally': 0.0016991163582915957, 'skin': 0.001474129006278047, 'isn': 0.0013986705392548678, 'fingers': 0.001396001737600

In [ ]:
print("Predictions for random forest 1000 of 2 sample leaves and for the review 1")
con_indx = list(feature_importance_grid[feature_importance_grid['ids'].isin(test_ids[random_review])].index)
print("Prediction: ", np.argmax(preds_grid[1]))
print("True Label: " + str(test_df['label'][1]))
print('' + "\nFeature contributions:")
negative = {}
positive = {}
for i in con_indx:
  c = contributions_grid[1][i]
  feature = feature_importance_grid['keys'][i]
  m = np.argmax(c)
  if m == 0:
    negative[feature] = list(c)
  else:
    positive[feature] = list(c)
print(''+ '\nthe ' + str(len(negative)) + ' contributions for negative are: ')
print({key: value[0] for key, value in sorted(negative.items(), key=lambda item: item[1], reverse=True)})
#for l in negative.:
 # print(str(l[0]) + '       has a contribution of: ' \
 #       + str(l[1]))
print(''+ '\nthe ' + str(len(positive)) + ' contributions for positive are: ')
for key, value in sorted(positive.items(), key=lambda item: item[1], reverse=True):
  print(key, value[1])

Predictions for random forest 1000 of 2 sample leaves and for the review 1
Prediction:  [0]
True Label: 0

Feature contributions:

the 7 contributions for negative are: 
{'hours': 0.011764360555364577, 'negative': 0.00027651255759575947, 'super': 0.00027072535321721403, '[CLS]': 0.0, '[SEP]': 0.0, 'body': 0.0, 'accord': 0.0}

the 9 contributions for positive are: 
honestly 1.4513613152756432e-05
rating 1.8691115099042525e-05
luck 4.79732854655969e-05
weird 6.249411893371428e-05
stuff 0.00010154275688092031
[PAD] 0.0025042998923106856
horrible 0.003057088536112245
rude 0.0031786743856853077
good 0.012982540852012014


In [ ]:
test_df.clean_text[1]

'clean hotel jacuzzi huge shower bathroom huge clean view nice hotel lobby nice valet great bad kept freezing hours minutes time floor heard train night thruout night loud clearly freight train engineer thats thing want waking away work makes nights broken sleep night good stay know vegas people hotel sleep'

In [ ]:
import random
random_review = random.randint(1, len(x_test))
print("Predictions for random forest 1000 of 2 sample leaves and for a random review")
print("Selected Sample     : %d"%random_review)
print("Actual Target Value : %s"%y_test[random_review])
print("Predicted Value     : %s"%np.argmax(preds_grid[random_review]))
print("" + "\nthe review is: " + "\n" + test_df.clean_text[random_review])
con_indx = list(feature_importance_grid[feature_importance_grid['ids'].isin(test_ids[random_review])].index)
print('' + "\nFeature contributions:")
negative = {}
positive = {}
for i in con_indx:
  c = contributions_grid[random_review][i]
  feature = feature_importance_grid['keys'][i]
  m = np.argmax(c)
  if m == 0:
    negative[feature] = list(c)
  else:
    positive[feature] = list(c)
print(''+ '\nthe ' + str(len(negative)) + ' contributions for negative are: ')
print({key: value[0] for key, value in sorted(negative.items(), key=lambda item: item[1], reverse=True)})
#for l in negative.:
 # print(str(l[0]) + '       has a contribution of: ' \
 #       + str(l[1]))
print(''+ '\nthe ' + str(len(positive)) + ' contributions for positive are: ')
for key, value in sorted(positive.items(), key=lambda item: item[1], reverse=True):
  print(key, value[1])

Predictions for random forest 1000 of 2 sample leaves and for a random review
Selected Sample     : 2581
Actual Target Value : 1
Predicted Value     : 1

the review is: 
drinks absolutely amazing food good chain good somewhat cooler weather type dishes expected english indian impossible meat dish amazing

Feature contributions:

the 13 contributions for negative are: 
{'expected': 0.019748940469908542, 'somewhat': 0.010087909875703379, 'indian': 0.007116088213652807, 'dish': 0.005234505167762018, 'food': 0.003634323628137821, 'english': 0.003483734482110269, 'drinks': 0.0029765395673710163, 'impossible': 0.002267870632705358, 'meat': 0.0022239553057390674, 'type': 0.00022125615323378273, '[CLS]': 0.0, '[SEP]': 0.0, 'cooler': 0.0}

the 7 contributions for positive are: 
weather 0.0006655086902850144
[PAD] 0.0013421150195339195
chain 0.0024021016907071198
absolutely 0.008481485402977761
dishes 0.018404713480581474
good 0.03556205369692527
amazing 0.1694398969017018


MAKE A FUNCTION FOR THE THING ABOVE SO IT CAN BE REPLICATED TO WHATEVER REVIEW, EITEHR RANDOM OF SPECIFIC AND THAT IT CAN BE SPECIFY IF YOU WANT RESULTS AS DICT OR LIST OR PRINTED ONLY... 

BABY THIS IS IT YOU CAN DO IT!

In [ ]:
test_df.columns

Index(['text', 'label', 'categories', 'clean_text'], dtype='object')

In [ ]:
test_df.label[0]

0

# Global features and
# Specific ones see if they match the reviews

## More about contributions

In [ ]:
#print("Bias For Sample 0                        : %s"%bias[0])
print("Constributions For Sample 0              : %s"%contributions_300[0])
print("Prediction Based on Bias & Contributions : %s"%(bias_300[0] + contributions_300[0].sum()))
print("Actual Target Value                      : %s"%y_test[0])
print("Target Value As Per Treeinterpreter      : %s"%preds_300[0][0])

Constributions For Sample 0              : [[-0.00218083  0.00218083]
 [ 0.          0.        ]
 [ 0.          0.        ]
 ...
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]]
Prediction Based on Bias & Contributions : [0.49902989 0.50097011]
Actual Target Value                      : 0
Target Value As Per Treeinterpreter      : 0.5157344272433114


In [ ]:
def create_contrbutions_df(contributions, random_sample, feature_names):
    contribs = contributions[random_sample].tolist()
    contribs.insert(0, bias[random_sample])
    contribs = np.array(contribs)
    contrib_df = pd.DataFrame(data=contribs, index=["Base"] + feature_names, columns=["Contributions_0", "Contributions_1"])
    prediction = contrib_df[["Contributions_0", "Contributions_1"]].sum()
    contrib_df.loc["Prediction"] = prediction
    return contrib_df

In [ ]:
import random
random_sample = random.randint(1, len(x_test))
print("Selected Sample     : %d"%random_sample)
print("Actual Target Value : %s"%y_test[random_sample])
print("Predicted Value     : %s"%np.argmax(preds[random_sample]))

Selected Sample     : 73
Actual Target Value : 0
Predicted Value     : 1


In [ ]:
# from: https://coderzcolumn.com/tutorials/machine-learning/treeinterpreter-interpreting-tree-based-models-prediction-of-individual-sample
contrib_df = create_contrbutions_df(contributions, random_sample, feature_names_test)
contrib_df


,Contributions_0,Contributions_1
Base,5.045256e-01,4.954744e-01
[CLS],0.000000e+00,0.000000e+00
[SEP],0.000000e+00,0.000000e+00
a,1.666848e-07,-1.666848e-07
d,-1.559995e-04,1.559995e-04
...,...,...
int,0.000000e+00,0.000000e+00
indifferent,0.000000e+00,0.000000e+00
amplified,0.000000e+00,0.000000e+00
vowed,0.000000e+00,0.000000e+00


In [ ]:
contrib_df.sort_values(by=['Contributions_0'], ascending=False)[:20]

,Contributions_0,Contributions_1
Base,0.504526,0.495474
Prediction,0.359114,0.640886
got,0.014659,-0.014659
great,0.014593,-0.014593
under,0.011655,-0.011655
how,0.011444,-0.011444
things,0.009973,-0.009973
only,0.007946,-0.007946
##s,0.006563,-0.006563
##co,0.005638,-0.005638


In [ ]:
test_df.text[73]

'i walked out after being ignored a friend of mine said the place was good but he is indian apparently a single white female doesn t rate to even get a glass of water when i was seated the host informed me it was a vegetarian restaurant ohkay i said fine with me and i smiled he turned on his heel as if it was a huge imposition to seat me i looked over the menu and waited for quite some time i closed the menu thinking that might bring the waiter i was disappointed i walked out and went to passage to india where i enjoyed their weekend buffet if you are a large group you might get some service i have no ideal how the food is but if you want to eat someone does have to serve you first i recommend going to any of the other dozen indian restaurants in the area instead '

In [ ]:
test_df.label[73]

0

In [ ]:
test_df.summa